In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer

stringIndexer = StringIndexer(inputCol="value", outputCol="valueIndex").setHandleInvalid("keep") 
encoder = OneHotEncoder(inputCol="valueIndex", outputCol="valueIndexVec")
pipeline = Pipeline(stages=[stringIndexer, encoder])
model = pipeline.fit(df)
transformed = model.transform(df)

In [4]:
sc = spark._jsc.sc()
sc.isStopped()

False

In [10]:
spark.getActiveSession()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()


In [13]:
configurations

[('spark.sql.catalogImplementation', 'hive'),
 ('spark.driver.host', 'Chensy.mshome.net'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.driver.port', '55345'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.name', 'PySparkShell'),
 ('spark.app.id', 'local-1644412018575'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.startTime', '1644412017588')]

In [11]:
configurations = 
for item in configurations:
    print(item)

('spark.sql.catalogImplementation', 'hive')
('spark.driver.host', 'Chensy.mshome.net')
('spark.rdd.compress', 'True')
('spark.serializer.objectStreamReset', '100')
('spark.master', 'local[*]')
('spark.driver.port', '55345')
('spark.submit.pyFiles', '')
('spark.executor.id', 'driver')
('spark.submit.deployMode', 'client')
('spark.app.name', 'PySparkShell')
('spark.app.id', 'local-1644412018575')
('spark.ui.showConsoleProgress', 'true')
('spark.app.startTime', '1644412017588')


[('spark.sql.catalogImplementation', 'hive'),
 ('spark.driver.host', 'Chensy.mshome.net'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.driver.port', '55345'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.name', 'PySparkShell'),
 ('spark.app.id', 'local-1644412018575'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.startTime', '1644412017588')]

In [3]:
spark._jsc.sc().isStopped()

False

# Dataset

1. 是Dataframe API的一个扩展，是Spark最新的数据抽象
1. 用户友好的API风格，既具有类型安全检查也具有Dataframe的查询优化特性。
1. Dataset支持编解码器，当需要访问非堆上的数据时可以避免反序列化整个对象，提高了效率。
1. 样例类被用来在Dataset中定义数据的结构信息，样例类中每个属性的名称直接映射到DataSet中的字段名称。
1. Dataframe是Dataset的特列，DataFrame=Dataset[Row] ，所以可以通过as方法将Dataframe转换为Dataset。Row是一个类型，跟Car、Person这些的类型一样，所有的表结构信息我都用Row来表示。
1. DataSet是强类型的。比如可以有Dataset[Car]，Dataset[Person].

DataFrame只是知道字段，但是不知道字段的类型，所以在执行这些操作的时候是没办法在编译的时候检查是否类型失败的，比如你可以对一个String进行减法操作，在执行的时候才报错，而DataSet不仅仅知道字段，而且知道字段类型，所以有更严格的错误检查。就跟JSON对象和类对象之间的类比。

- 在对DataFrame和Dataset进行操作许多操作都需要这个包进行支持：import spark.implicits._
- DataFrame和Dataset均可使用模式匹配获取各个字段的值和类型

#### DataFrame

```scala
testDF.map{
      case Row(col1:String,col2:Int)=>
        println(col1);println(col2)
        col1
      case _=>
        ""
    }
```

# OutOfMemoryError

> Bug1： java.lang.OutOfMemoryError: GC overhead limit exceeded

> Bug2：java.lang.OutOfMemoryError: Java heap space

原因分析：

1. collect 导致数据回传 Driver，致使 Driver 内存溢出

    - 解决方法是增加 Driver 内存

2. executor-core 数量太多，导致多个 core 之间争夺 GC 时间以及资源，最后导致大部分时间都花在了 GC 上

    - 解决方法是减少 core 数量

```python
# - V.1
conf = SparkConf()
conf.set("spark.app.name", "es-hadoop")
conf.set("spark.executor.memory", "64g")
conf.set("spark.master", "spark://192.168.50.21:7077")
conf.set("spark.executor.cores", "4")
conf.set("spark.driver.memory", "5g")  # 这里是增加jvm的内存
conf.set("spark.driver.maxResultSize", "2g") # 这里是最大显示结果，这里是提示我改的。

# - V.2
# jvm分配的内存空间不足,  把相应的内存设置大点即可.
spark = SparkSession \

    .builder \

    .appName("Python Spark SQL basic example") \

    .config('spark.executor.memory','8g')\

    .config('spark.driver.memory','8g')\

    .config('spark.driver.maxResultsSize','0')\

    .getOrCreate()
```

---